In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [32]:
pair = "EURCHF"
NB_H1_CANDLES = 20000

In [33]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [34]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an = apply_patterns(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
df_an.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
0,2020-01-02 09:00:00,1.08715,1.08751,1.08682,1.08721,1.08715,1.08721,47.826087,56.521739,8.695652,...,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-02 10:00:00,1.08722,1.08752,1.08619,1.08619,1.08619,1.08722,0.000000,77.443609,77.443609,...,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2020-01-02 11:00:00,1.08619,1.08650,1.08495,1.08607,1.08607,1.08619,72.258065,80.000000,7.741935,...,8.695652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-02 12:00:00,1.08606,1.08617,1.08486,1.08598,1.08598,1.08606,85.496183,91.603053,6.106870,...,77.443609,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-02 13:00:00,1.08598,1.08643,1.08563,1.08584,1.08584,1.08598,26.250000,43.750000,17.500000,...,7.741935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_an = RSI(df_an)
df_an = BollingerBands(df_an)
df_an = ATR(df_an)
df_an = KeltnerChannels(df_an)
df_an = MACD(df_an)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,BB_MA,BB_UP,BB_LW,ATR_14,EMA,KeUp,KeLo,MACD,SIGNAL_MACD,HIST
0,2020-01-02 09:00:00,1.08715,1.08751,1.08682,1.08721,1.08715,1.08721,47.826087,56.521739,8.695652,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02 10:00:00,1.08722,1.08752,1.08619,1.08619,1.08619,1.08722,0.000000,77.443609,77.443609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-02 11:00:00,1.08619,1.08650,1.08495,1.08607,1.08607,1.08619,72.258065,80.000000,7.741935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-02 12:00:00,1.08606,1.08617,1.08486,1.08598,1.08598,1.08606,85.496183,91.603053,6.106870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-02 13:00:00,1.08598,1.08643,1.08563,1.08584,1.08584,1.08598,26.250000,43.750000,17.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,2023-03-23 11:00:00,0.99613,0.99907,0.99513,0.99668,0.99613,0.99668,25.380711,39.340102,13.959391,...,0.997123,0.999079,0.995167,0.001547,0.997025,1.000681,0.993369,0.000678,0.000809,-0.000132
19995,2023-03-23 12:00:00,0.99667,0.99778,0.99546,0.99715,0.99667,0.99715,52.155172,72.844828,20.689655,...,0.997189,0.998996,0.995382,0.001602,0.997037,1.000941,0.993133,0.000607,0.000769,-0.000162
19996,2023-03-23 13:00:00,0.99714,0.99736,0.99611,0.99688,0.99688,0.99714,61.600000,82.400000,20.800000,...,0.997214,0.998981,0.995446,0.001632,0.997022,1.000854,0.993190,0.000523,0.000720,-0.000197
19997,2023-03-23 14:00:00,0.99687,0.99747,0.99655,0.99676,0.99676,0.99687,22.826087,34.782609,11.956522,...,0.997226,0.998982,0.995470,0.001660,0.996997,1.000883,0.993111,0.000442,0.000664,-0.000222


In [36]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST'],
      dtype='object')

In [37]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.0001

In [38]:
df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)
df_an

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,KeLo,MACD,SIGNAL_MACD,HIST,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next
0,2020-01-03 18:00:00,1.08352,1.08496,1.08348,1.08395,1.08352,1.08395,2.702703,31.756757,29.054054,...,1.081101,-0.000256,-0.000324,0.000067,False,False,1.08331,1.08519,1.08393,1.08455
1,2020-01-03 19:00:00,1.08395,1.08455,1.08393,1.08450,1.08395,1.08450,3.225806,91.935484,88.709677,...,1.081394,-0.000194,-0.000295,0.000101,False,False,1.08348,1.08496,1.08432,1.08573
2,2020-01-03 20:00:00,1.08450,1.08573,1.08432,1.08536,1.08450,1.08536,12.765957,73.758865,60.992908,...,1.081705,-0.000077,-0.000247,0.000170,False,True,1.08393,1.08455,1.08494,1.08561
3,2020-01-03 21:00:00,1.08539,1.08561,1.08494,1.08494,1.08494,1.08539,0.000000,67.164179,67.164179,...,1.081875,-0.000017,-0.000198,0.000180,False,False,1.08432,1.08573,1.08482,1.08537
4,2020-01-03 22:00:00,1.08492,1.08537,1.08482,1.08497,1.08492,1.08497,18.181818,27.272727,9.090909,...,1.082061,0.000032,-0.000149,0.000181,False,False,1.08494,1.08561,1.08425,1.08501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19941,2023-03-23 10:00:00,0.99908,0.99948,0.99347,0.99612,0.99612,0.99908,44.093178,93.344426,49.251248,...,0.994099,0.000806,0.000842,-0.000036,True,True,0.99839,0.99940,0.99513,0.99907
19942,2023-03-23 11:00:00,0.99613,0.99907,0.99513,0.99668,0.99613,0.99668,25.380711,39.340102,13.959391,...,0.993369,0.000678,0.000809,-0.000132,False,False,0.99347,0.99948,0.99546,0.99778
19943,2023-03-23 12:00:00,0.99667,0.99778,0.99546,0.99715,0.99667,0.99715,52.155172,72.844828,20.689655,...,0.993133,0.000607,0.000769,-0.000162,False,False,0.99513,0.99907,0.99611,0.99736
19944,2023-03-23 13:00:00,0.99714,0.99736,0.99611,0.99688,0.99688,0.99714,61.600000,82.400000,20.800000,...,0.993190,0.000523,0.000720,-0.000197,False,False,0.99546,0.99778,0.99655,0.99747


In [39]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0

for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    df_an['SL'][i] = df_an.low[i-1] #- SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i] #+ SPREAD
                    break
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    df_an['SL'][i] = df_an.high[i-1] #+ SPREAD
                    df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_4528\1003418580.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = SELL
C:\Users\meysam\AppData\Local\Temp\ipykernel_4528\1003418580.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.high[i-1] #+ SPREAD
C:\Users\meysam\AppData\Local\Temp\ipykernel_4528\1003418580.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_an.close

In [40]:
df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

In [45]:
df_an.to_csv(f"data/{pair}.csv")

In [42]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST', 'pivots_l', 'pivots_h', 'low_perv', 'high_perv',
       'low_next', 'high_next', 'SIGNAL', 'TP', 'SL'],
      dtype='object')

In [43]:
X = []
Y = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces = []
for i in range(4,len(df_an)):
#     if df_an.SIGNAL[i] != NONE:
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [44]:
print(len(X))
print(len(Y))
print(len(indeces))

8513
8513
8513


In [14]:
print(len(feature))

202


In [15]:
SPLIT_INDEX = int(0.8*len(X))

In [16]:

X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]


In [17]:
print(len(X_train))
print(len(Y_train))
print(len(indeces_train))
print(len(X_test))
print(len(Y_test))
print(len(indeces_test))

6915
6915
6915
1729
1729
1729


In [18]:
# scaler = StandardScaler()  
# scaler.fit(X_train)  
# X_train = scaler.transform(X_train)  
# >>> # apply same transformation to test data
# >>> X_test = scaler.transform(X_test) 


In [19]:
# help(MLPClassifier)

In [20]:
clf = MLPClassifier( solver='adam', alpha=1e-5,learning_rate_init=1e-5,
                     tol=0.0000010,
                    hidden_layer_sizes=(200,200), 
                    random_state=1,max_iter=100000,verbose=True,warm_start = True)
# clf = MLPClassifier(learning_rate_init=1e-6, warm_start = True, random_state = 2,hidden_layer_sizes= (60,40),max_iter=100000,verbose=True, tol=0.00000010,n_iter_no_change=40)

In [21]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 11.07300861
Iteration 2, loss = 7.71000667
Iteration 3, loss = 4.99574002
Iteration 4, loss = 3.70407992
Iteration 5, loss = 3.23534529
Iteration 6, loss = 2.92818896
Iteration 7, loss = 2.63785337
Iteration 8, loss = 2.36627214
Iteration 9, loss = 2.11581587
Iteration 10, loss = 1.89436443
Iteration 11, loss = 1.71219078
Iteration 12, loss = 1.57004583
Iteration 13, loss = 1.46450272
Iteration 14, loss = 1.38968578
Iteration 15, loss = 1.34009230
Iteration 16, loss = 1.30044248
Iteration 17, loss = 1.26991360
Iteration 18, loss = 1.24557096
Iteration 19, loss = 1.22623505
Iteration 20, loss = 1.20745091
Iteration 21, loss = 1.19296826
Iteration 22, loss = 1.17732822
Iteration 23, loss = 1.16516313
Iteration 24, loss = 1.15175278
Iteration 25, loss = 1.14247729
Iteration 26, loss = 1.12849944
Iteration 27, loss = 1.11807853
Iteration 28, loss = 1.10963576
Iteration 29, loss = 1.09879235
Iteration 30, loss = 1.09026022
Iteration 31, loss = 1.08086033
Iteration 32, lo

Iteration 253, loss = 0.72518856
Iteration 254, loss = 0.72617148
Iteration 255, loss = 0.72514537
Iteration 256, loss = 0.72349386
Iteration 257, loss = 0.72275796
Iteration 258, loss = 0.72265430
Iteration 259, loss = 0.72122655
Iteration 260, loss = 0.71979509
Iteration 261, loss = 0.72069376
Iteration 262, loss = 0.71934355
Iteration 263, loss = 0.72216069
Iteration 264, loss = 0.72222362
Iteration 265, loss = 0.71933635
Iteration 266, loss = 0.71616049
Iteration 267, loss = 0.71851127
Iteration 268, loss = 0.71558936
Iteration 269, loss = 0.72041048
Iteration 270, loss = 0.71658712
Iteration 271, loss = 0.71512858
Iteration 272, loss = 0.71666835
Iteration 273, loss = 0.71645889
Iteration 274, loss = 0.71488461
Iteration 275, loss = 0.71447358
Iteration 276, loss = 0.71471457
Iteration 277, loss = 0.71340779
Iteration 278, loss = 0.71420056
Iteration 279, loss = 0.71232444
Iteration 280, loss = 0.71226467
Iteration 281, loss = 0.71152341
Iteration 282, loss = 0.70969624
Iteration 

Iteration 502, loss = 0.63597138
Iteration 503, loss = 0.63708738
Iteration 504, loss = 0.63662933
Iteration 505, loss = 0.63515918
Iteration 506, loss = 0.63425543
Iteration 507, loss = 0.63416386
Iteration 508, loss = 0.63346331
Iteration 509, loss = 0.63205246
Iteration 510, loss = 0.63215596
Iteration 511, loss = 0.63371092
Iteration 512, loss = 0.63192940
Iteration 513, loss = 0.63479262
Iteration 514, loss = 0.63133169
Iteration 515, loss = 0.63081894
Iteration 516, loss = 0.63236894
Iteration 517, loss = 0.63426913
Iteration 518, loss = 0.63070976
Iteration 519, loss = 0.63001908
Iteration 520, loss = 0.63156484
Iteration 521, loss = 0.63000116
Iteration 522, loss = 0.63018154
Iteration 523, loss = 0.63068475
Iteration 524, loss = 0.62807559
Iteration 525, loss = 0.63027065
Iteration 526, loss = 0.62857669
Iteration 527, loss = 0.62855530
Iteration 528, loss = 0.62885821
Iteration 529, loss = 0.62985818
Iteration 530, loss = 0.62732888
Iteration 531, loss = 0.62674980
Iteration 

Iteration 751, loss = 0.57491107
Iteration 752, loss = 0.57346606
Iteration 753, loss = 0.57241767
Iteration 754, loss = 0.57228775
Iteration 755, loss = 0.57186937
Iteration 756, loss = 0.57245050
Iteration 757, loss = 0.57252555
Iteration 758, loss = 0.57283492
Iteration 759, loss = 0.57296774
Iteration 760, loss = 0.57373148
Iteration 761, loss = 0.57373817
Iteration 762, loss = 0.57111357
Iteration 763, loss = 0.57067858
Iteration 764, loss = 0.57024145
Iteration 765, loss = 0.57073170
Iteration 766, loss = 0.57210674
Iteration 767, loss = 0.57260576
Iteration 768, loss = 0.57211428
Iteration 769, loss = 0.57254494
Iteration 770, loss = 0.57332067
Iteration 771, loss = 0.56915852
Iteration 772, loss = 0.56901400
Iteration 773, loss = 0.56972502
Iteration 774, loss = 0.56965616
Iteration 775, loss = 0.56816504
Iteration 776, loss = 0.56992870
Iteration 777, loss = 0.56779967
Iteration 778, loss = 0.57225054
Iteration 779, loss = 0.57060930
Iteration 780, loss = 0.56771938
Iteration 

Iteration 1000, loss = 0.52621225
Iteration 1001, loss = 0.52606038
Iteration 1002, loss = 0.52394906
Iteration 1003, loss = 0.52348479
Iteration 1004, loss = 0.52335537
Iteration 1005, loss = 0.52287079
Iteration 1006, loss = 0.52194958
Iteration 1007, loss = 0.52265573
Iteration 1008, loss = 0.52255367
Iteration 1009, loss = 0.52246706
Iteration 1010, loss = 0.52192044
Iteration 1011, loss = 0.52394726
Iteration 1012, loss = 0.52702730
Iteration 1013, loss = 0.52178488
Iteration 1014, loss = 0.52318455
Iteration 1015, loss = 0.52237717
Iteration 1016, loss = 0.52174065
Iteration 1017, loss = 0.52359035
Iteration 1018, loss = 0.52004436
Iteration 1019, loss = 0.52059611
Iteration 1020, loss = 0.52123426
Iteration 1021, loss = 0.52139930
Iteration 1022, loss = 0.52039837
Iteration 1023, loss = 0.51996089
Iteration 1024, loss = 0.52012609
Iteration 1025, loss = 0.51921732
Iteration 1026, loss = 0.51947084
Iteration 1027, loss = 0.51934661
Iteration 1028, loss = 0.51939916
Iteration 1029

Iteration 1241, loss = 0.48215095
Iteration 1242, loss = 0.48163707
Iteration 1243, loss = 0.48137202
Iteration 1244, loss = 0.48123385
Iteration 1245, loss = 0.48246386
Iteration 1246, loss = 0.48295110
Iteration 1247, loss = 0.48030236
Iteration 1248, loss = 0.48178000
Iteration 1249, loss = 0.48031560
Iteration 1250, loss = 0.47994201
Iteration 1251, loss = 0.47975368
Iteration 1252, loss = 0.47942660
Iteration 1253, loss = 0.47883312
Iteration 1254, loss = 0.47990002
Iteration 1255, loss = 0.48147022
Iteration 1256, loss = 0.48563814
Iteration 1257, loss = 0.47969552
Iteration 1258, loss = 0.47837683
Iteration 1259, loss = 0.47869969
Iteration 1260, loss = 0.47953842
Iteration 1261, loss = 0.47865152
Iteration 1262, loss = 0.47995393
Iteration 1263, loss = 0.47900927
Iteration 1264, loss = 0.47714447
Iteration 1265, loss = 0.47743643
Iteration 1266, loss = 0.47760091
Iteration 1267, loss = 0.47859697
Iteration 1268, loss = 0.47761197
Iteration 1269, loss = 0.47773015
Iteration 1270

Iteration 1482, loss = 0.44401463
Iteration 1483, loss = 0.44330615
Iteration 1484, loss = 0.44413494
Iteration 1485, loss = 0.44403496
Iteration 1486, loss = 0.44373754
Iteration 1487, loss = 0.44553067
Iteration 1488, loss = 0.44549112
Iteration 1489, loss = 0.44773861
Iteration 1490, loss = 0.44358959
Iteration 1491, loss = 0.44322573
Iteration 1492, loss = 0.44648846
Iteration 1493, loss = 0.44429736
Iteration 1494, loss = 0.44351089
Iteration 1495, loss = 0.44231706
Iteration 1496, loss = 0.44243442
Iteration 1497, loss = 0.44213497
Iteration 1498, loss = 0.44235413
Iteration 1499, loss = 0.44350314
Iteration 1500, loss = 0.44513917
Iteration 1501, loss = 0.44253807
Iteration 1502, loss = 0.44266016
Iteration 1503, loss = 0.44155160
Iteration 1504, loss = 0.44132564
Iteration 1505, loss = 0.44041171
Iteration 1506, loss = 0.44079220
Iteration 1507, loss = 0.44082142
Iteration 1508, loss = 0.43977431
Iteration 1509, loss = 0.44119499
Iteration 1510, loss = 0.44007175
Iteration 1511

Iteration 1723, loss = 0.41080819
Iteration 1724, loss = 0.41094428
Iteration 1725, loss = 0.40988458
Iteration 1726, loss = 0.40994503
Iteration 1727, loss = 0.41059796
Iteration 1728, loss = 0.41182830
Iteration 1729, loss = 0.40961243
Iteration 1730, loss = 0.41012341
Iteration 1731, loss = 0.40931050
Iteration 1732, loss = 0.40850567
Iteration 1733, loss = 0.40864986
Iteration 1734, loss = 0.40943651
Iteration 1735, loss = 0.40894490
Iteration 1736, loss = 0.40923506
Iteration 1737, loss = 0.41036838
Iteration 1738, loss = 0.40774073
Iteration 1739, loss = 0.40821867
Iteration 1740, loss = 0.40872697
Iteration 1741, loss = 0.40969663
Iteration 1742, loss = 0.40802703
Iteration 1743, loss = 0.40879782
Iteration 1744, loss = 0.40778398
Iteration 1745, loss = 0.40763665
Iteration 1746, loss = 0.40803170
Iteration 1747, loss = 0.40813470
Iteration 1748, loss = 0.40856196
Iteration 1749, loss = 0.41026013
Iteration 1750, loss = 0.40671133
Iteration 1751, loss = 0.40671159
Iteration 1752

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(200, 200),
              learning_rate_init=1e-05, max_iter=100000, random_state=1,
              tol=1e-06, verbose=True, warm_start=True)

In [22]:
y_perdict = clf.predict(X_test)

In [23]:
confusion_matrix(Y_test, y_perdict)

array([[134, 211,   4],
       [203, 656, 190],
       [  1, 201, 129]], dtype=int64)

In [24]:
# pickle.dump(clf, open(f"{pair}.pickle", "wb"))

In [25]:
(Y_test==y_perdict).sum()/len(y_perdict)

0.5315211104684789

In [26]:
print(len(indeces_test))
print(len(y_perdict))
# y_perdict

1729
1729


In [27]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'body_lower', 'body_upper',
       'body_bottom_perc', 'body_top_perc', 'body_perc', 'direction',
       'body_size', 'low_change', 'high_change', 'body_size_change',
       'mid_point', 'mid_point_prev_2', 'body_size_prev', 'direction_prev',
       'direction_prev_2', 'body_perc_prev', 'body_perc_prev_2', 'HANGING_MAN',
       'SHOOTING_STAR', 'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR', 'RSI_14', 'BB_MA',
       'BB_UP', 'BB_LW', 'ATR_14', 'EMA', 'KeUp', 'KeLo', 'MACD',
       'SIGNAL_MACD', 'HIST', 'pivots_l', 'pivots_h', 'low_perv', 'high_perv',
       'low_next', 'high_next', 'SIGNAL', 'TP', 'SL'],
      dtype='object')

In [28]:
indeces_test[0]

16001

In [29]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
        if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\4200224076.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.SIGNAL.loc[idx] = y_perdict[i]


In [30]:
df_test

,time,open,high,low,close,TP,SL,pivots_l,pivots_h,SIGNAL
0,2022-08-02 07:00:00,1.22567,1.22619,1.22432,1.22479,1.22165,1.22636,0.0,0.0,0
1,2022-08-02 08:00:00,1.22479,1.22551,1.22384,1.22435,0.00000,0.00000,0.0,0.0,0
2,2022-08-02 09:00:00,1.22435,1.22435,1.22181,1.22198,0.00000,0.00000,0.0,0.0,0
3,2022-08-02 10:00:00,1.22198,1.22218,1.21884,1.22006,0.00000,0.00000,0.0,0.0,0
4,2022-08-02 11:00:00,1.22006,1.22054,1.21849,1.21906,0.00000,0.00000,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
3961,2023-03-23 10:00:00,1.23105,1.23286,1.23045,1.23206,1.22750,1.23434,1.0,0.0,0
3962,2023-03-23 11:00:00,1.23206,1.23319,1.23088,1.23153,0.00000,0.00000,0.0,1.0,0
3963,2023-03-23 12:00:00,1.23154,1.23162,1.22841,1.22886,0.00000,0.00000,0.0,0.0,0
3964,2023-03-23 13:00:00,1.22886,1.23070,1.22822,1.23026,0.00000,0.00000,0.0,0.0,0


In [31]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51558 entries, 0 to 51557
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    51558 non-null  datetime64[ns]
 1   open    51558 non-null  float64       
 2   high    51558 non-null  float64       
 3   low     51558 non-null  float64       
 4   close   51558 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 2.0 MB


In [32]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [33]:
df_res_m5 = gt.run_test()

run_test...
Result:
win rate:  0.615506329113924
 2.0    389
-1.0    243
Name: result, dtype: int64


In [62]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1186191000.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1186191000.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [63]:
df_res_m5

,running,start_index_m5,profit_factor,loss_factor,SPREAD,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,balance,time
0,False,2226,2,-1.0,0.00004,1.03511,1.03553,-1,0.00000,0.00000,-1.0,2022-11-28 08:00:00,2022-11-28 08:00:00,97.000000,2022-11-28 08:00:00
1,False,2238,2,-1.0,0.00004,1.03722,1.04321,1,1.04216,1.03469,2.0,2022-11-28 10:35:00,2022-11-28 09:00:00,103.889684,2022-11-28 10:35:00
2,False,2513,2,-1.0,0.00004,1.03868,1.03683,-1,1.03714,1.03945,2.0,2022-11-29 08:30:00,2022-11-29 08:00:00,111.377836,2022-11-29 08:30:00
3,False,2537,2,-1.0,0.00004,1.03817,1.03813,1,0.00000,0.00000,2.0,2022-11-29 10:00:00,2022-11-29 10:00:00,111.377965,2022-11-29 10:00:00
4,False,2585,2,-1.0,0.00004,1.03826,1.03603,-1,1.03606,1.03936,2.0,2022-11-29 14:45:00,2022-11-29 14:00:00,118.151770,2022-11-29 14:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,False,25001,2,-1.0,0.00004,1.07677,1.07710,-1,0.00000,0.00000,-1.0,2023-03-21 19:00:00,2023-03-21 19:00:00,2527.456233,2023-03-21 19:00:00
312,False,25108,2,-1.0,0.00004,1.07730,1.07748,1,0.00000,0.00000,2.0,2023-03-22 04:00:00,2023-03-22 04:00:00,2527.468902,2023-03-22 04:00:00
313,False,25132,2,-1.0,0.00004,1.07702,1.07699,1,0.00000,0.00000,2.0,2023-03-22 06:00:00,2023-03-22 06:00:00,2527.471014,2023-03-22 06:00:00
314,False,25168,2,-1.0,0.00004,1.07666,1.07721,-1,0.00000,0.00000,-1.0,2023-03-22 09:00:00,2023-03-22 09:00:00,2451.646883,2023-03-22 09:00:00


In [64]:
df_res_m5.result.value_counts()

 2.0    202
-1.0    114
Name: result, dtype: int64

In [65]:
df_res_m5_plot = df_res_m5#.loc[:200]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])

In [38]:
#  cross test
pair_cross = "EURUSD"
NB_H1_CANDLES_CROSS = 2000
rates = Rates(pair_cross, NB_H1_CANDLES_CROSS, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an_cross = rates.get_rates_from_now()
df_an_cross.drop(NB_H1_CANDLES_CROSS-1,inplace=True) 
df_an_cross.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']

df_an_cross = apply_patterns(df_an_cross)
for cp in candle_patterns:
    df_an_cross[cp] = df_an_cross[cp].astype(float)

df_an_cross = RSI(df_an_cross)
df_an_cross = BollingerBands(df_an_cross)
df_an_cross = ATR(df_an_cross)
df_an_cross = KeltnerChannels(df_an_cross)
df_an_cross = MACD(df_an_cross)

df_an_cross['pivots_l'] = False
df_an_cross['pivots_h'] = False

df_an_cross['low_perv'] = df_an_cross.low.shift(1)
df_an_cross['high_perv'] = df_an_cross.high.shift(1)
df_an_cross['low_next'] = df_an_cross.low.shift(-1)
df_an_cross['high_next'] = df_an_cross.high.shift(-1)
df_an_cross['pivots_l'] = (df_an_cross.low_perv > df_an_cross.low) & (df_an_cross.low_next > df_an_cross.low)
df_an_cross['pivots_h'] = (df_an_cross.high_perv < df_an_cross.high) & (df_an_cross.high_next < df_an_cross.high)
df_an_cross.dropna(inplace=True)
df_an_cross.reset_index(drop=True, inplace=True)

df_an_cross.head()

,time,open,high,low,close,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,KeLo,MACD,SIGNAL_MACD,HIST,pivots_l,pivots_h,low_perv,high_perv,low_next,high_next
0,2022-11-25 17:00:00,1.03815,1.04070,1.03764,1.04067,1.03815,1.04067,16.666667,99.019608,82.352941,...,1.034932,-0.000531,-0.000390,-0.000141,False,False,1.03549,1.03886,1.03917,1.04114
1,2022-11-25 18:00:00,1.04067,1.04114,1.03917,1.04014,1.04014,1.04067,49.238579,76.142132,26.903553,...,1.034860,-0.000462,-0.000406,-0.000057,False,True,1.03764,1.04070,1.03990,1.04094
2,2022-11-25 19:00:00,1.04013,1.04094,1.03990,1.04059,1.04013,1.04059,22.115385,66.346154,44.230769,...,1.034971,-0.000370,-0.000398,0.000028,False,False,1.03917,1.04114,1.03969,1.04061
3,2022-11-25 20:00:00,1.04060,1.04061,1.03969,1.04016,1.04016,1.04060,51.086957,98.913043,47.826087,...,1.035189,-0.000325,-0.000382,0.000057,True,False,1.03990,1.04094,1.03988,1.04150
4,2022-11-25 21:00:00,1.04016,1.04150,1.03988,1.04118,1.04016,1.04118,17.283951,80.246914,62.962963,...,1.035883,-0.000209,-0.000346,0.000136,False,True,1.03969,1.04061,1.04077,1.04142


In [39]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

4e-05

In [40]:
BUY = 1
SELL = -1
NONE = 0

df_an_cross['SIGNAL'] = 0
df_an_cross['TP'] = 0
df_an_cross['SL'] = 0

for i in range(1,len(df_an_cross)):
    if df_an_cross.pivots_l[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.low[j] <= df_an_cross.low[i-1] ):
                break
            else:
                if(df_an_cross.high[j] > 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = BUY
                    df_an_cross['SL'][i] = df_an_cross.low[i-1] #- SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.low[i-1] ) + df_an_cross.close[i] #+ SPREAD
                    break
    if df_an_cross.pivots_h[i-1]:
        for j in range(i,len(df_an_cross)):
            if( df_an_cross.high[j] >= df_an_cross.high[i-1] ):
                break
            else:
                if(df_an_cross.low[j] < 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]):
#                     df_an_cross.SIGNAL[i] = SELL
                    df_an_cross['SL'][i] = df_an_cross.high[i-1] #+ SPREAD
                    df_an_cross['TP'][i] = 2*(df_an_cross.close[i] - df_an_cross.high[i-1] ) + df_an_cross.close[i]
                    break

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\3328372866.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\3328372866.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
df_an_cross.pivots_l = df_an_cross.pivots_l.astype(float)
df_an_cross.pivots_h = df_an_cross.pivots_h.astype(float)

In [42]:
X_cross = []
Y_cross = [] 
feature_col = [
    'open', 'high', 'low', 'close',
               'RSI_14', 'ATR_14',
    'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
    'MACD', 'SIGNAL_MACD', 'HIST',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]
indeces_cross = []
for i in range(4,len(df_an_cross)):
    if df_an_cross.pivots_h[i-1] or df_an_cross.pivots_l[i-1]:
#         print("im here")
        feature = []
        feature.append(df_an_cross.pivots_h[i-1])
        feature.append(df_an_cross.pivots_l[i-1])
        for fc in feature_col:
            feature.append(df_an_cross[fc][i])
            feature.append(df_an_cross[fc][i-1])
            feature.append(df_an_cross[fc][i-2])
            feature.append(df_an_cross[fc][i-3])
            feature.append(df_an_cross[fc][i-4])

        X_cross.append(feature)
        Y_cross.append(df_an_cross.SIGNAL[i])
        indeces_cross.append(i)

In [43]:
len(feature)

202

In [44]:
# clf = pickle.load(open("GBPUSD.pickle", "rb"))

In [45]:
y_perdict_cross = clf.predict(X_cross)

In [46]:
confusion_matrix(Y_cross, y_perdict_cross)

array([[  0,   0,   0],
       [175, 477, 162],
       [  0,   0,   0]], dtype=int64)

In [47]:
(Y_cross==y_perdict_cross).sum()/len(y_perdict_cross)

0.585995085995086

In [48]:
df_test = df_an_cross[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']]#.iloc[indeces_cross[0]-5:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_cross):
    if i > 0:
        if df_test.pivots_l[idx-1] and y_perdict_cross[i] == BUY:
            df_test.SIGNAL.loc[idx] = y_perdict_cross[i]
        if df_test.pivots_h[idx-1] and y_perdict_cross[i] == SELL:
#         if df_test.pivots_h[idx-1] or df_test.pivots_l[idx-1] :
            df_test.SIGNAL.loc[idx] = y_perdict_cross[i]

df_test.reset_index(drop=True, inplace=True)


C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\732658451.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\732658451.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\732658451.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
rates = Rates(pair_cross, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    25545 non-null  datetime64[ns]
 1   open    25545 non-null  float64       
 2   high    25545 non-null  float64       
 3   low     25545 non-null  float64       
 4   close   25545 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 998.0 KB


In [50]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [51]:
df_res_m5 = gt.run_test()

run_test...
Result:
win rate:  0.6392405063291139
 2.0    202
-1.0    114
Name: result, dtype: int64


In [52]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1186191000.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7280\1186191000.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [53]:
df_res_m5_plot = df_res_m5#.loc[:200]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
cp.show_plot(line_traces=['balance'])